<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#판다스-형" data-toc-modified-id="판다스-형-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>판다스 형</a></span></li><li><span><a href="#Numpy-형" data-toc-modified-id="Numpy-형-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Numpy 형</a></span></li></ul></div>

In [18]:
import pandas as pd
import time

df = pd.read_excel('./500매트릭스.xlsx', index_col=0)
df2 = pd.read_csv('./500종점.csv')
df3 = pd.read_csv('./500기점.csv')

start_time = time.time()

wntO = df2['종점'].tolist()
wntD = df3['기점'].tolist()

df_copy = df.copy()
value = df_copy.shape[0]

df[value] = pd.Series(wntO) # 시리즈 변환
df.loc[value] = pd.Series(wntD) # .loc 는 row

df['Ei'] = df[value]/df[value-1] 
df.loc['Fj'] = df.loc[value]/df.loc[value-1]

df = df.fillna(0)
apple = []

# 유출통행량 보정값

for i in range(len(df)-3):
    value2 = df.iloc[i,value-1]/sum(df.loc[i] * df.loc['Fj'])
    apple.append(value2)

df['Li'] = pd.Series(apple)
df = df.fillna(0)

banana = []

# 유입통행량 보정값

for i in range(len(df)-3):
    value3 = df.loc[value-1,i]/sum(df[i]*df['Ei'])
    banana.append(value3)

df.loc['Lj'] = pd.Series(banana)
df = df.fillna(0)

end_time = time.time()
execution_time = end_time - start_time

print('실행시간:',execution_time)

실행시간: 0.2540585994720459


## 판다스 형

In [19]:
start_time = time.time()

result = df_copy.applymap(lambda x: None)
value = df_copy.shape[0]

i = ''
j = ''

for i in range(value-1):
    for j in range(value-1):
        #print(i, ',', j)
        result.loc[i,j] = df.loc[i,j]*df.loc[i,'Ei']*df.loc['Fj'][j]*(df.loc[i,'Li'] + df.loc['Lj'][j])/2
    
end_time = time.time()
execution_time = end_time - start_time

print('실행시간:',execution_time)

실행시간: 56.050780296325684


## Numpy 형

In [20]:
import numpy as np
start_time = time.time()

result = df_copy.applymap(lambda x: None)
value = df_copy.shape[0]

i = ''
j = ''
# 데이터프레임을 NumPy 배열로 변환
df_values = df.values
result = np.zeros((value, value))  # 결과를 저장할 NumPy 배열

for i in range(value):
    for j in range(value):
        result[i, j] = df_values[i, j] * df_values[i, value+1] * df_values[value+1, j] * (df_values[i, value+2] + df_values[value+2, j]) / 2
end_time = time.time()
execution_time = end_time - start_time

print('실행시간:',execution_time)

실행시간: 0.5232255458831787


와.. 실행시간이 50배 이상 줄었다

In [21]:
df_values

array([[1.00000000e+00, 6.00000000e+00, 1.00000000e+00, ...,
        1.20200000e+03, 5.54939982e-01, 4.57330622e-01],
       [9.00000000e+00, 2.00000000e+00, 7.00000000e+00, ...,
        6.62800000e+03, 2.95233853e+00, 4.52084901e-01],
       [9.00000000e+00, 3.00000000e+00, 5.00000000e+00, ...,
        3.24800000e+03, 1.45259392e+00, 4.37150226e-01],
       ...,
       [6.40300000e+03, 3.84400000e+03, 3.62700000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.76110392e+00, 1.74410163e+00, 1.68933395e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.54119253e-01, 4.54346200e-01, 4.41637929e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])